## User_Group (사용자 집단별 추천)
- 사용자들을 비슷한 특성(성별, 나이, 직업 등)의 소집단으로 분류한 후 소집단별 평점평균

In [84]:
import numpy as np
import pandas as pd

### Data Load

In [85]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.user', sep='|', names=u_cols, encoding='latin-1')
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [86]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies.head()

,movie_id,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [87]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [88]:
# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### movie data 재구성 : [movie_id, title]
- 사용자 집단별 추천을 위해  movie data를 재구성

In [89]:
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### train, test set 분리
- user_id를 기준으로 일정 비율(stratify = true)로 학습, 테스트 데이터 분리

In [90]:
from sklearn.model_selection import train_test_split
X = ratings.copy()
y = ratings['user_id'] # 스칼라값
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y) # user_id로 stratify한다.

In [91]:
X_train

,user_id,movie_id,rating
41817,44,67,3
4069,49,531,3
90857,705,117,5
1608,130,901,1
98676,929,431,1
...,...,...,...
81140,823,170,4
10337,331,64,4
73532,501,100,4
79661,284,319,3


In [106]:
X_test

,user_id,movie_id,rating
13932,405,440,1
45536,94,201,4
47079,679,357,5
96396,825,1049,3
29606,458,521,4
...,...,...,...
97994,738,250,4
53693,648,498,3
7677,178,480,3
94169,500,49,4


In [92]:
y_test

13932    405
45536     94
47079    679
96396    825
29606    458
        ... 
97994    738
53693    648
7677     178
94169    500
96842    882
Name: user_id, Length: 25000, dtype: int64

### 평가 방법 (성능 측정)
- RMSE 성능측정 : $\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y_i})^2}$
- 평균 제곱근 오차를 사용하는 이유 : 영화의 예측 평점과 실제 평점 간의 차이를 평가하는 지표로 자주 사용 ( = 예측 값이 연속적인 경우에 잘 맞는 평가 지표, 영화 평점처럼 숫자로 된 값을 다루는 상황에서 적합한 평가지표 )
    - 예측 오차를 직관적으로 측정 : RMSE는 예측된 평점과 실제 평점이 얼마나 일치하는지를 직관적으로 평가할 수 있다.
    - 큰 오차에 더 큰 페널티 부여 : RMSE는 오차의 제곱을 구하기 때문에 큰 오차에 대해 더 큰 페널티를 부여한다. (= 큰 오차를 줄이는 데 초점을 맞추는 평가지표)
    - 연속적인 평점 예측에 적합 : 영화 추천에서 평점은 대게 연속적인 값 (ex : 1~5점)으로 나타나므로 분류문제가 아닌 회귀문제에 가깝다.
    - 해석의 용이성 : 실제 평점 단위와 돌일한 단위를 가지기 때문에 해석에 용이하다.

In [93]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

### 모델에 따른 테스트 데이터의 예측 및 실데이터 간의 정확도 계산

In [94]:
# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(X_test['user_id'], X_test['movie_id']) # iterable (tuple로 생성)
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs]) # 평점의 배열
    y_true = np.array(X_test['rating']) # 실제로 본 영화의 배열
    return RMSE(y_true, y_pred)

### train data (사용자 X 영화 X 평점) matrix

In [95]:
# train 데이터로 Full matrix 구하기 
rating_matrix = X_train.pivot(index='user_id', columns='movie_id', values='rating')
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1670,1672,1673,1674,1675,1677,1678,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### train data에서 사용자별 영화 평점평균 추출

In [96]:
# 전체 평균으로 예측치를 계산하는 기본 모델 - 영화 평점 평균
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

- 위 함수는 영화 추천 시스템에서 평점 예측을 단순히 전체 평균에 기반해 계산하는 기본 모델이다.
- 사용자가 특정 영화에 대해 평점을 부여할때, 과거의 영화 평점 평균을 사용하여 예측하는 방식

In [97]:
# 학습데이터에서의 영화별 평점평균
train_mean = X_train.groupby(['movie_id'])['rating'].mean()
train_mean.head()

movie_id
1    3.879412
2    3.212121
3    3.016393
4    3.586667
5    3.338235
Name: rating, dtype: float64

- X_train 데이터프레임에서 영화별로 평균 평점을 계산하고, 그 결과의 일부를 출력하는 과정
    - `X_train.groupby(['movie_id'])`: movie_id를 기준으로 그룹화
    - `['rating'].mean()`: rating 컬럼의 평균값을 계산

In [98]:
# 테스트데이터 (사용자 x 영화): 사용자가 평가한 영화(실데이터)와 사용자가 평가한 영화의 학습데이터에서의 평점평균간 오차
print(score(best_seller))

1.0268630873093618


---
## 사용자 집단별 (성별) 추천

In [99]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### user_id를 기준으로 users table과 학습데이터 (rating) 병합

In [101]:
merged_ratings = pd.merge(X_train, users, on='user_id')
users = users.set_index('user_id')
merged_ratings.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,44,67,3,26,M,technician,46260
1,49,531,3,23,F,student,76111
2,705,117,5,21,F,student,54494
3,130,901,1,20,M,none,60115
4,929,431,1,44,M,scientist,53711


In [102]:
users

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [103]:
# gender별 평점평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
g_mean.head()

movie_id  sex
1         F      3.758621
          M      3.920949
2         F      3.235294
          M      3.207317
3         F      2.750000
Name: rating, dtype: float64

- `merged_ratings[['movie_id', 'sex', 'rating']]` : 해당 열을 선택하여 리스트로 전달
- `.groupby(['movie_id', 'sex'])` : movie_id와 sex열을 기준으로 데이터를 그룹화
    - 그룹화는 동일한 값들을 가진 행들을 묶어주는 역할을 한다.

### 성별로 평가한 영화의 평점 평균

In [104]:
# gender별 평균을 예측치로 돌려주는 함수 - 평가자 성별
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex'] # 사용자의 성별 가져오기
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender] # 성별별 평균 평점 확인
        else:
            gender_rating = 3.0 # 기본값
    else:
        gender_rating = 3.0
    return gender_rating

- `.loc[user_id]` : 인덱스가 user_id인 행을 선택
    - `users.loc[user_id]['sex']` : user_id의 sex열의 값을 가져온다.
    - `.loc[]` : 레이블 기반의 인덱싱을 제공하는 판다스의 데이터 선택 메서드


In [105]:
# 테스트데이터 (사용자 x 영화): 사용자의 평가한 영화(실데이터)와 학습데이터에서 성별로 평가한 영화의 평점평균간 오차
print('테스트 데이터 점수 : ', score(cf_gender))

테스트 데이터 점수 :  1.0374576424297546


---
## 사용자 직업별 추천 결과는?
## 사용자 [성별, 직업별] 추천 결과는?

In [109]:
merged_ratings.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,44,67,3,26,M,technician,46260
1,49,531,3,23,F,student,76111
2,705,117,5,21,F,student,54494
3,130,901,1,20,M,none,60115
4,929,431,1,44,M,scientist,53711


In [112]:
merged_ratings['occupation'].value_counts()

occupation
student          16469
other             7997
educator          7084
engineer          6130
programmer        5849
administrator     5608
writer            4153
librarian         3953
technician        2630
executive         2549
healthcare        2105
artist            1733
entertainment     1571
scientist         1545
marketing         1461
retired           1208
lawyer            1010
none               676
salesman           641
doctor             404
homemaker          224
Name: count, dtype: int64

In [115]:
users

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [114]:
# occupation별 평점평균 계산
o_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()
o_mean.head()

movie_id  occupation   
1         administrator    4.043478
          artist           3.900000
          doctor           3.500000
          educator         3.612903
          engineer         4.028571
Name: rating, dtype: float64

### 직업별 평가한 영화의 평점평균

In [117]:
def cf_occupation(user_id, movie_id):
    if movie_id in rating_matrix:
        occupation = users.loc[user_id]['occupation']
        if occupation in o_mean[movie_id]:
            occupation_rating = o_mean[movie_id][occupation]
        else:
            occupation_rating = 3.0
    else:
        occupation_rating = 3.0
    return occupation_rating    
            

In [118]:
print('테스트 데이터 점수 : ', score(cf_occupation))

테스트 데이터 점수 :  1.1217564142864747


### 코드 개선

In [119]:
def cf_occupation(user_id, movie_id):
    # 사용자 ID 존재 여부 확인
    if user_id not in users.index:
        return 3.0  # 또는 적절한 기본값

    occupation = users.loc[user_id]['occupation']

    # 영화 ID 존재 여부 확인
    if movie_id in o_mean:
        # 해당 영화의 직업별 평균 평점 가져오기
        movie_occupations = o_mean[movie_id]
        # 사용자의 직업이 해당 영화에 대한 직업별 평균에 있는지 확인
        if occupation in movie_occupations:
            occupation_rating = movie_occupations[occupation]
        else:
            occupation_rating = 3.0  # 또는 전체 평균 평점
    else:
        occupation_rating = 3.0  # 또는 전체 평균 평점

    return occupation_rating

- 위 작성된 함수 코드 개선
    - 초기 코드에서 rating_matrix에서 movie_id의 존재여부를 확인하는 부분이 부적절.
    - 사용자 ID와 영화 ID의 존재 여부를 확인하고, 각 단계에서 발생할 수 있는 예외를 처리하여 함수의 안정성을 높였다.

In [120]:
print('테스트 데이터 점수 : ', score(cf_occupation))

테스트 데이터 점수 :  1.1217564142864747


- 결과 분석
    - RMSE는 예측값과 실제값 사이의 평균적인 오차를 나타냄. 1.12라는 것은 예측값이 실제값과 평균적으로 1.12정도 차이가 난다는 것을 의미.
    - 현재 RMSE점수만으로는 좋은 점수라고 보기 어렵다. 개선의 여지가 있으며 다른 모델들과의 비교를 통해 상대적인 성능을 평가하는 것이 중요하다.